In [5]:
import os
import csv
from xml.etree.ElementTree import Element, SubElement, ElementTree

def create_xml_annotation(image_filename, bbox_data):
    annotation = Element('annotation')

    folder = SubElement(annotation, 'folder')
    folder.text = 'images'  # Set your image folder name here

    filename = SubElement(annotation, 'filename')
    filename.text = image_filename

    size = SubElement(annotation, 'size')
    width = SubElement(size, 'width')
    width.text = str(bbox_data['width'])
    height = SubElement(size, 'height')
    height.text = str(bbox_data['height'])

    for bbox in bbox_data['bboxes']:
        object_elem = SubElement(annotation, 'object')
        name = SubElement(object_elem, 'name')
        name.text = bbox['class']
        pose = SubElement(object_elem, 'pose')
        pose.text = 'Unspecified'
        truncated = SubElement(object_elem, 'truncated')
        truncated.text = '0'
        difficult = SubElement(object_elem, 'difficult')
        difficult.text = '0'

        bndbox = SubElement(object_elem, 'bndbox')
        xmin = SubElement(bndbox, 'xmin')
        xmin.text = str(bbox['xmin'])
        ymin = SubElement(bndbox, 'ymin')
        ymin.text = str(bbox['ymin'])
        xmax = SubElement(bndbox, 'xmax')
        xmax.text = str(bbox['xmax'])
        ymax = SubElement(bndbox, 'ymax')
        ymax.text = str(bbox['ymax'])

    return ElementTree(annotation)

# CSV file path
csv_file = 'data.csv'

# Read CSV file
with open(csv_file, mode='r') as file:
    csv_reader = csv.DictReader(file)
    
    # Initialize a dictionary to store bounding box data for each image
    image_bbox_data = {}
    
    for row in csv_reader:
        image_filename = row['filename']
        image_width = int(row['width'])
        image_height = int(row['height'])

        bbox = {
            'class': row['class'],
            'xmin': int(row['xmin']),
            'ymin': int(row['ymin']),
            'xmax': int(row['xmax']),
            'ymax': int(row['ymax'])
        }
        
        # If the image is not in the dictionary, add it with the initial bounding box data
        if image_filename not in image_bbox_data:
            image_bbox_data[image_filename] = {
                'width': image_width,
                'height': image_height,
                'bboxes': [bbox]
            }
        else:
            # If the image is already in the dictionary, append the new bounding box data
            image_bbox_data[image_filename]['bboxes'].append(bbox)

# Create XML annotations for each image
for image_filename, bbox_data in image_bbox_data.items():
    xml_tree = create_xml_annotation(image_filename, bbox_data)
    xml_filename = os.path.splitext(image_filename)[0] + '.xml'
    xml_path = os.path.join('annotations', xml_filename)  # Save XML files in the 'annotations' directory
    xml_tree.write(xml_path)

print('Conversion complete.')


Conversion complete.
